# 导入包

In [1]:
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2

# 构造参数解析器并解析参数

In [2]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--images", type=str,
                help="path to input directory of images to stitch")
ap.add_argument("-o", "--output", type=str,
                help="path to the output image")
ap.add_argument("-c", "--crop", type=int, default=1,
                help="whether to crop out largest rectangular region")
arg_list = ['-i', 'image', '-o', 'output.png', '-c', '1']
args = vars(ap.parse_args(arg_list))

# 加载图像

In [3]:
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["images"])))
images = []
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    images.append(image)
print("[INFO] stitching images...")
stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)

[INFO] loading images...
[INFO] stitching images...


# 如果状态为 '0'，则成功执行图像拼接

In [4]:
if status == 0:
    if args["crop"] > 0:
        print("[INFO] cropping...")
        stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,
                                      cv2.BORDER_CONSTANT, (0, 0, 0))
        #将拼接后的图像转换为灰度并设置阈值
        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

        #查找最大等值线
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # 为掩码分配内存
        mask = np.zeros(thresh.shape, dtype="uint8")
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

        # 创建两个mask：一个作为我们的实际最小矩形区域和另一个用作计数器的区域
        minRect = mask.copy()
        sub = mask.copy()


        while cv2.countNonZero(sub) > 0:
            # 继续循环，直到 没有非零像素留在
            minRect = cv2.erode(minRect, None)
            sub = cv2.subtract(minRect, thresh)

        # 提取边界框 （x， y） 坐标
        cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)

        # 使用边界框坐标提取我们的最终拼接图像
        stitched = stitched[y:y + h, x:x + w]

    # 将输出拼接后的图片写入磁盘
    cv2.imwrite(args["output"], stitched)

    # 输出
    cv2.imshow("Stitched", stitched)
    cv2.waitKey(0)

[INFO] cropping...


# 否则拼接失败，可能是因为关键点不够被检测到

In [5]:
else:
    print("[INFO] image stitching failed ({})".format(status))

SyntaxError: invalid syntax (3226929527.py, line 1)